# Model selection

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.metrics import root_mean_squared_error as rmse
from xgboost import XGBRegressor

from utils import get_gain_ranking

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.3f}'.format)

## 1. Data import

In [3]:
DATA_DIR = 'data'

In [4]:
TRAIN_FILE_NAME = 'train_dev.csv'

In [5]:
train_file_path = os.path.join(DATA_DIR, TRAIN_FILE_NAME)

In [6]:
df = pd.read_csv(train_file_path).set_index('user_id')

In [7]:
df.shape

(4862, 83)

In [8]:
df.head()

,entry_point=android,country_e,country=US,battles_won_d0,battles_won_d1,battles_won_d3,battles_won_d7,battles_lost_d0,battles_lost_d1,battles_lost_d3,battles_lost_d7,battles_win_ratio_d0,battles_win_ratio_d1,battles_win_ratio_d3,battles_win_ratio_d7,session_time_d0,session_time_d1,session_time_d3,session_time_d7,session_mean_d0,session_mean_d1,session_mean_d3,session_mean_d7,n_sessions_d0,n_sessions_d1,n_sessions_d3,n_sessions_d7,inactive_d1,n_active_days,wealth_on_login_max_d0,wealth_on_login_max_d1,wealth_on_login_max_d3,wealth_on_login_max_d7,wealth_on_login_min_d0,wealth_on_login_min_d1,wealth_on_login_min_d3,wealth_on_login_min_d7,wealth_growth_d0_d1,wealth_growth_d1_d3,wealth_growth_d3_d7,wealth_growth_ratio_d0_d1,wealth_growth_ratio_d1_d3,wealth_growth_ratio_d3_d7,finish_quest_sum_d0,finish_quest_sum_d1,finish_quest_sum_d3,finish_quest_sum_d7,n_finish_quest_40_d0,n_finish_quest_40_d1,n_finish_quest_40_d3,n_finish_quest_40_d7,n_finish_quest_50_d0,n_finish_quest_50_d1,n_finish_quest_50_d3,n_finish_quest_50_d7,level_up_max_d0,level_up_max_d1,level_up_max_d3,level_up_max_d7,levels_unique_d0,levels_unique_d1,levels_unique_d3,levels_unique_d7,payment_sum_d0,payment_sum_d1,payment_sum_d3,payment_sum_d7,payment_max_d0,payment_max_d1,payment_max_d3,payment_max_d7,payment_mean_d0,payment_mean_d1,payment_mean_d3,payment_mean_d7,n_payments_d0,n_payments_d1,n_payments_d3,n_payments_d7,wealth_on_login_max_d0=802,wealth_on_login_max_d7=802,segment,ltv_30
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,3,1,47.000,172.000,214.000,232.000,1.000,1.000,2.000,8.000,0.000,0.000,0.000,0.000,341.000,761.000,1192.000,1921.000,20.059,19.025,12.817,10.977,17,40,93,175,0,8,20695.000,186656.000,249194.000,419827.000,802.000,802.000,802.000,802.000,165961.000,62538.000,170633.000,0.183,0.210,0.336,270,400,1150,1740,3,4,9,13,0,0,1,1,10,17,17,19,9.000,16.000,16.000,18.000,122.255,195.608,195.608,391.363,24.451,24.451,24.451,97.902,24.451,24.451,24.451,32.614,5.000,8.000,8.000,12.000,0,0,TRAIN,1468.187
2,1,8,0,0.000,34.000,69.000,153.000,0.000,1.000,1.000,10.000,0.000,0.000,0.000,0.000,33.000,188.000,598.000,1453.000,6.600,18.800,26.000,25.946,5,10,23,56,0,8,806.000,9045.000,28680.000,90769.000,802.000,802.000,802.000,802.000,8239.000,19635.000,62089.000,0.009,0.028,0.088,0,0,390,1230,0,0,2,9,0,0,1,2,2,9,14,15,1.000,8.000,13.000,14.000,0.000,0.000,30.648,30.648,0.000,0.000,30.648,30.648,0.000,0.000,30.648,30.648,NaN,NaN,1.000,1.000,0,0,TRAIN,619.849
3,1,3,1,41.000,65.000,78.000,113.000,13.000,20.000,20.000,23.000,0.000,0.000,0.000,0.000,505.000,883.000,1106.000,1567.000,72.143,55.188,46.083,38.220,7,16,24,41,0,8,4684.000,15214.000,32761.000,58584.000,802.000,802.000,802.000,802.000,10530.000,17547.000,25823.000,0.015,0.032,0.057,230,340,600,970,2,2,3,4,0,0,1,1,9,13,14,16,8.000,12.000,13.000,15.000,0.000,48.902,48.902,73.353,0.000,24.451,24.451,24.451,0.000,24.451,24.451,24.451,NaN,2.000,2.000,3.000,0,0,TRAIN,97.804
4,1,13,0,26.000,47.000,89.000,125.000,1.000,1.000,3.000,3.000,0.000,0.000,0.000,0.000,126.000,222.000,631.000,860.000,42.000,44.400,45.071,45.263,3,5,14,19,0,7,8940.000,12900.000,40856.000,82547.000,802.000,802.000,802.000,802.000,3960.000,27956.000,41691.000,0.013,0.040,0.079,100,260,780,1240,1,2,6,7,0,0,0,1,9,9,14,16,8.000,8.000,13.000,15.000,0.000,0.000,71.662,71.662,0.000,0.000,23.906,23.906,0.000,0.000,23.887,23.887,NaN,NaN,3.000,3.000,0,0,TRAIN,71.662
5,1,8,0,44.000,67.000,94.000,127.000,2.000,2.000,2.000,2.000,0.000,0.000,0.000,0.000,535.000,1075.000,1766.000,2523.000,53.500,43.000,30.982,25.745,10,25,57,98,0,8,17689.000,25283.000,32983.000,39425.000,802.000,802.000,802.000,802.000,7594.000,7700.000,6442.000,0.025,0.032,0.038,350,590,980,1830,1,4,5,8,1,1,1,3,9,10,10,10,8.000,9.000,9.000,9.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN,NaN,NaN,NaN,0,0,TRAIN,0.000


## 2. Datasets preparation

In [9]:
TARGET = 'ltv_30'

In [10]:
(df[TARGET] == 0).mean()

np.float64(0.637186343068696)

In [11]:
features = [x for x in df.columns.tolist() if x not in ['segment', TARGET]]

In [12]:
train_df = df[df['segment'] == 'TRAIN']
dev_df = df[df['segment'] == 'DEV']

In [13]:
print(train_df.shape)
print(dev_df.shape)

(3889, 83)
(973, 83)


In [14]:
X_train = train_df[features]#.values
y_train = train_df[TARGET].values

X_dev = dev_df[features]#.values
y_dev = dev_df[TARGET].values

In [15]:
print(X_train.shape)
print(y_train.shape)
print()
print(X_dev.shape)
print(y_dev.shape)

(3889, 81)
(3889,)

(973, 81)
(973,)


## 3. Model selection

In [16]:
n_xgb = 20

In [17]:
XGB_HYPERPARAMS_FIXED = {
    'booster': 'gbtree',
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'n_jobs': -1,
    
    'random_state': 42,
}

In [18]:
xgb_df = pd.DataFrame()

In [19]:
xgb_df['n_estimators'] = np.random.randint(low=100, high=1001, size=n_xgb)
xgb_df['max_depth'] = np.random.randint(low=3, high=11, size=n_xgb)
xgb_df['learning_rate'] = 3 * 10 ** np.random.uniform(low=-2.477, high=-1, size=n_xgb)
xgb_df['subsample'] = 10 ** np.random.uniform(low=-0.181, high=0, size=n_xgb)
xgb_df['colsample_bytree'] = np.random.uniform(low=0.5, high=1.0, size=n_xgb)
xgb_df['min_child_weight'] = np.random.uniform(low=1, high=10, size=n_xgb)
xgb_df['gamma'] = np.random.uniform(low=0, high=5, size=n_xgb)
xgb_df['reg_alpha'] = np.random.uniform(low=0, high=10, size=n_xgb)
xgb_df['reg_lambda'] = np.random.uniform(low=0, high=10, size=n_xgb)

In [20]:
xgb_df

,n_estimators,max_depth,learning_rate,subsample,colsample_bytree,min_child_weight,gamma,reg_alpha,reg_lambda
0,773,10,0.012,0.949,0.779,3.301,4.875,6.134,0.285
1,726,10,0.025,0.750,0.842,5.392,1.306,7.814,1.446
2,876,9,0.169,0.768,0.741,1.338,0.116,5.268,4.803
3,859,9,0.161,0.703,0.738,4.853,2.751,0.194,9.082
4,715,3,0.088,0.896,0.837,8.347,4.479,6.481,4.331
5,275,4,0.122,0.897,0.641,3.841,0.222,0.690,6.905
6,909,3,0.017,0.715,0.517,2.159,3.189,9.972,8.499
7,157,6,0.032,0.898,0.902,1.459,1.673,7.059,1.429
8,257,8,0.097,0.738,0.947,3.854,3.986,8.882,1.553
9,950,5,0.026,0.952,0.681,2.476,4.713,6.476,3.612


In [21]:
for index, item in xgb_df.iterrows():

    hp_var = {
        'n_estimators': int(item['n_estimators']),
        'max_depth': int(item['max_depth']),
        'learning_rate': item['learning_rate'],
        'subsample': item['subsample'],
        'colsample_bytree': item['colsample_bytree'],
        'min_child_weight': item['min_child_weight'],
        'gamma': item['gamma'],
        'reg_alpha': item['reg_alpha'],
        'reg_lambda': item['reg_lambda'],
    }

    print(hp_var)

    hp = XGB_HYPERPARAMS_FIXED | hp_var
    
    model = XGBRegressor(**hp)

    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_dev_pred = model.predict(X_dev)
    
    xgb_df.loc[index, 'rmse_train'] = rmse(y_train, y_train_pred)
    xgb_df.loc[index, 'rmse_dev'] = rmse(y_dev, y_dev_pred)

{'n_estimators': 773, 'max_depth': 10, 'learning_rate': np.float64(0.011888508106264763), 'subsample': np.float64(0.9492620533826412), 'colsample_bytree': np.float64(0.7788803502410688), 'min_child_weight': np.float64(3.3008528622135387), 'gamma': np.float64(4.874800927214214), 'reg_alpha': np.float64(6.133881409754791), 'reg_lambda': np.float64(0.2853806254999569)}
{'n_estimators': 726, 'max_depth': 10, 'learning_rate': np.float64(0.02470642552246917), 'subsample': np.float64(0.7498149858304566), 'colsample_bytree': np.float64(0.8419787314731617), 'min_child_weight': np.float64(5.392489381355348), 'gamma': np.float64(1.3059287331070473), 'reg_alpha': np.float64(7.814389979837656), 'reg_lambda': np.float64(1.4458586924703576)}
{'n_estimators': 876, 'max_depth': 9, 'learning_rate': np.float64(0.16904524992078435), 'subsample': np.float64(0.7684227966285097), 'colsample_bytree': np.float64(0.7410646435895123), 'min_child_weight': np.float64(1.3384372409424414), 'gamma': np.float64(0.1162

In [22]:
xgb_df = xgb_df.sort_values(by='rmse_dev', ascending=True).reset_index(drop=True)

In [23]:
xgb_df.style.format({
    'learning_rate': '{:.4f}',
    'subsample': '{:.3f}',
    'colsample_bytree': '{:.3f}',
    'min_child_weight': '{:.3f}',
    'gamma': '{:.3f}',
    'reg_alpha': '{:.3f}',
    'reg_lambda': '{:.3f}',
    'scale_pos_weight': '{:.3f}',
    
    'rmse_train': '{:.2f}',
    'rmse_dev': '{:.2f}',
}).background_gradient(axis=0, cmap='YlOrRd', subset=['learning_rate', 'subsample'])

,n_estimators,max_depth,learning_rate,subsample,colsample_bytree,min_child_weight,gamma,reg_alpha,reg_lambda,rmse_train,rmse_dev
0,157,6,0.0321,0.898,0.902,1.459,1.673,7.059,1.429,130.15,429.81
1,950,5,0.0260,0.952,0.681,2.476,4.713,6.476,3.612,70.10,437.67
2,452,8,0.0135,0.760,0.647,4.101,3.518,0.665,8.633,278.23,442.11
3,909,3,0.0173,0.715,0.517,2.159,3.189,9.972,8.499,259.64,450.67
4,595,8,0.0174,0.915,0.863,7.393,4.527,9.336,6.456,185.47,457.99
5,726,10,0.0247,0.750,0.842,5.392,1.306,7.814,1.446,79.52,467.80
6,257,8,0.0966,0.738,0.947,3.854,3.986,8.882,1.553,32.91,468.95
7,376,8,0.0503,0.705,0.587,6.541,0.615,6.456,3.521,125.75,468.99
8,884,5,0.0142,0.785,0.617,8.974,1.376,3.646,2.408,210.49,472.91
9,234,9,0.0581,0.699,0.731,4.422,4.041,7.072,4.177,137.07,473.24


In [26]:
xgb_df.loc[0].to_dict()

{'n_estimators': 157.0,
 'max_depth': 6.0,
 'learning_rate': 0.03210921617155426,
 'subsample': 0.8979614161067629,
 'colsample_bytree': 0.9024503388694571,
 'min_child_weight': 1.4590048269858573,
 'gamma': 1.6730631959514086,
 'reg_alpha': 7.0586731716217646,
 'reg_lambda': 1.4294052894285947,
 'rmse_train': 130.15102571385802,
 'rmse_dev': 429.81041807913454}

In [25]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

## 4. Feature selection

In [27]:
BEST_HP = {
    'n_estimators': 157,
    'max_depth': 6,
    'learning_rate': 0.03210921617155426,
    'subsample': 0.8979614161067629,
    'colsample_bytree': 0.9024503388694571,
    'min_child_weight': 1.4590048269858573,
    'gamma': 1.6730631959514086,
    'reg_alpha': 7.0586731716217646,
    'reg_lambda': 1.4294052894285947,
}

In [28]:
hp = XGB_HYPERPARAMS_FIXED | BEST_HP

In [29]:
best_model = XGBRegressor(**hp)

In [30]:
best_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9024503388694571, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric='rmse', feature_types=None, feature_weights=None,
             gamma=1.6730631959514086, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.03210921617155426,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=1.4590048269858573, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=157,
             n_jobs=-1, num_parallel_tree=None, ...)

In [32]:
get_gain_ranking(model).head(20)

,gain
n_payments_d7,0.295
wealth_on_login_min_d0,0.142
payment_sum_d7,0.062
payment_max_d7,0.061
n_payments_d0,0.040
levels_unique_d1,0.036
n_finish_quest_50_d3,0.035
n_payments_d3,0.025
payment_mean_d3,0.023
payment_mean_d7,0.023


In [35]:
get_gain_ranking(model).head(30).index.tolist()

['n_payments_d7',
 'wealth_on_login_min_d0',
 'payment_sum_d7',
 'payment_max_d7',
 'n_payments_d0',
 'levels_unique_d1',
 'n_finish_quest_50_d3',
 'n_payments_d3',
 'payment_mean_d3',
 'payment_mean_d7',
 'payment_mean_d1',
 'levels_unique_d7',
 'session_mean_d1',
 'wealth_growth_d3_d7',
 'n_finish_quest_50_d7',
 'n_finish_quest_40_d7',
 'battles_lost_d7',
 'level_up_max_d0',
 'wealth_on_login_max_d3',
 'levels_unique_d3',
 'wealth_growth_ratio_d1_d3',
 'session_mean_d7',
 'n_sessions_d1',
 'wealth_on_login_max_d0',
 'n_finish_quest_40_d0',
 'session_mean_d0',
 'battles_lost_d3',
 'wealth_on_login_max_d1',
 'n_finish_quest_50_d1',
 'battles_won_d7']